## **Task 2**

In [5]:
# to access google drive folder
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

train_X = np.load("/content/drive/MyDrive/train_x.npy", allow_pickle=True)
train_y = np.load("/content/drive/MyDrive/train_y.npy", allow_pickle=True)

validate_X = np.load("/content/drive/MyDrive/valid_x.npy", allow_pickle=True)
validate_y = np.load("/content/drive/MyDrive/valid_y.npy", allow_pickle=True)

test_x = np.load("/content/drive/MyDrive/test_x.npy", allow_pickle=True)
test_y = np.load("/content/drive/MyDrive/test_y.npy", allow_pickle=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import matplotlib.pyplot as plt

def image_normalization(arr):
    return (arr - arr.min())/(arr.max()-arr.min())

def disable_ax_ticks(ax):
    ax.set_xticks([])
    ax.set_xticks([], minor=True)
    ax.set_yticks([])
    ax.set_yticks([], minor=True)

def show_mnist_examples(x, y):
    fig = plt.figure(constrained_layout=True,figsize=(12,9), dpi=100)
    gs = fig.add_gridspec(3,4)
    main_ax = fig.add_subplot(gs[:3,:3])
    fig.suptitle(y)
    #main_ax.imshow(np.moveaxis(x, 0, -1))# .imshow(x)
    main_ax.imshow(image_normalization(np.moveaxis(x, 0, -1)))
    disable_ax_ticks(main_ax)

    for j in range(3):
      c_ax = fig.add_subplot(gs[j,-1])
      subimage = x.copy()
      subimage[:j] = 0
      subimage[j+1:] = 0
      subimage[j] = subimage[j]-subimage[j].min()
      c_ax.imshow(image_normalization(np.moveaxis(subimage, 0, -1)))
      disable_ax_ticks(c_ax)
    plt.show()

# separating the color channels and reprocessing subimages
def separate_image(image):
  subimage_A = image[0]
  subimage_B = image[1]
  subimage_C = image[2]

  subimage_A = preprocess_image(subimage_A)
  subimage_B = preprocess_image(subimage_B)
  subimage_C = preprocess_image(subimage_C)

  return subimage_A, subimage_B, subimage_C

# scale to [0, 1] and reshape to (28, 28, 1)
def preprocess_model_data(image):
  image = image.astype("float32") / 255
  return np.expand_dims(image, -1)

# normalize and reshape to (28, 28, 1)
def preprocess_image(image):
  image = image_normalization(image)
  return np.expand_dims(image, -1)

# adding frame for width and height 32
def add_frame(image):
  return np.pad(image, ((0, 0), (2,2), (2, 2)), 'constant', constant_values=(0))




# Pre-processing the data for model

In [7]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Resizing
from tensorflow.keras.models import Sequential
import numpy as np

num_classes = 20

batch_size = 95
epochs = 20

trainX = np.array([add_frame(image) for image in train_X])
validateX = np.array([add_frame(image) for image in validate_X])
testX = np.array([add_frame(image) for image in test_x])

trainX = np.resize(trainX, (trainX.shape[0], 32, 32, 3))
validateX = np.resize(validateX, (validateX.shape[0], 32, 32, 3))
testX = np.resize(testX, (testX.shape[0], 32, 32, 3))

trainX = preprocess_image(trainX)
validateX = preprocess_image(validateX)
testX = preprocess_image(testX)

trainY = keras.utils.to_categorical(train_y, num_classes)
validateY = keras.utils.to_categorical(validate_y, num_classes)
testY = keras.utils.to_categorical(test_y, num_classes)


# Xception

In [ ]:
from tensorflow.keras.applications import Xception
epochs = 20

modelXc = Sequential()

base_model = Xception(include_top=False, weights=None, classes=num_classes, pooling="max")

modelXc.add(Resizing(72, 72, interpolation="bilinear", input_shape=(32, 32, 3)))
modelXc.add(base_model)
modelXc.add(Flatten())
modelXc.add(Dense(512, activation='relu'))
modelXc.add(Dense(20, activation='softmax'))

modelXc.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

print(modelXc.summary())

history = modelXc.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelXc.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 72, 72, 3)         0         
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 20)                10260     
                                                                 
Total params: 21,920,828
Trainable params: 21,866,300
Non-trainable params: 54,528
_________________________________________________________________
None
Epoch 1/20
474/474 [==============

# VGG16

In [ ]:
from tensorflow.keras.applications import VGG16

epochs = 15

modelVGG = Sequential()

base_model = VGG16(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelVGG.add(base_model)
modelVGG.add(Flatten())
modelVGG.add(Dense(512, activation='relu'))
modelVGG.add(Dense(20, activation='softmax'))

modelVGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

history = modelVGG.fit(trainX, trainY, epochs=epochs, batch_size=90, validation_data=(validateX, validateY))

results = modelVGG.evaluate(testX, testY, batch_size=90)


Epoch 1/15


KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.applications import VGG16

epochs = 15

modelVGG = Sequential()

base_model = VGG16(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelVGG.add(base_model)
modelVGG.add(Flatten())
modelVGG.add(Dense(512, activation='relu'))
modelVGG.add(Dense(20, activation='softmax'))

modelVGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

modelVGG.summary()

history = modelVGG.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelVGG.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)               262656    
                                                                 
 dense_7 (Dense)             (None, 20)                10260     
                                                                 
Total params: 14,987,604
Trainable params: 14,987,604
Non-trainable params: 0
_________________________________________________________________


# ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50

epochs = 20

modelRN50 = Sequential()

base_model = ResNet50(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelRN50.add(base_model)
modelRN50.add(Flatten())
modelRN50.add(Dense(512, activation='relu'))
modelRN50.add(Dense(20, activation='softmax'))

modelRN50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

history = modelRN50.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelRN50.evaluate(testX, testY, batch_size=batch_size)

Epoch 1/20
474/474 [==============================] - 82s 153ms/step - loss: 2.6300 - accuracy: 0.1352 - val_loss: 3.8874 - val_accuracy: 0.0704
Epoch 2/20
474/474 [==============================] - 69s 145ms/step - loss: 2.0632 - accuracy: 0.2324 - val_loss: 2.0623 - val_accuracy: 0.2456
Epoch 3/20
474/474 [==============================] - 69s 145ms/step - loss: 1.7540 - accuracy: 0.3244 - val_loss: 1.6451 - val_accuracy: 0.3872
Epoch 4/20
474/474 [==============================] - 69s 145ms/step - loss: 1.4615 - accuracy: 0.4347 - val_loss: 1.6152 - val_accuracy: 0.4417
Epoch 5/20
474/474 [==============================] - 74s 155ms/step - loss: 1.1958 - accuracy: 0.5466 - val_loss: 1.3930 - val_accuracy: 0.5592
Epoch 6/20
474/474 [==============================] - 69s 145ms/step - loss: 0.9597 - accuracy: 0.6545 - val_loss: 1.1015 - val_accuracy: 0.6731
Epoch 7/20
474/474 [==============================] - 69s 145ms/step - loss: 0.7179 - accuracy: 0.7584 - val_loss: 0.9013 - val_ac

# ResNet50V2

In [ ]:
from tensorflow.keras.applications import ResNet50V2

epochs = 20

modelRN502 = Sequential()

base_model = ResNet50V2(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelRN502.add(base_model)
modelRN502.add(Flatten())
modelRN502.add(Dense(512, activation='relu'))
modelRN502.add(Dense(20, activation='softmax'))

modelRN502.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy", ""])
print(modelRN502.summary())

history = modelRN502.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelRN502.evaluate(testX, testY, batch_size=batch_size)

Epoch 1/20
474/474 [==============================] - 77s 148ms/step - loss: 2.6927 - accuracy: 0.1251 - val_loss: 3.4932 - val_accuracy: 0.0693
Epoch 2/20
474/474 [==============================] - 74s 155ms/step - loss: 2.1194 - accuracy: 0.2196 - val_loss: 2.0646 - val_accuracy: 0.2499
Epoch 3/20
474/474 [==============================] - 74s 155ms/step - loss: 1.8294 - accuracy: 0.2979 - val_loss: 2.0691 - val_accuracy: 0.2322
Epoch 4/20
474/474 [==============================] - 69s 145ms/step - loss: 1.5705 - accuracy: 0.3895 - val_loss: 1.7358 - val_accuracy: 0.3723
Epoch 5/20
474/474 [==============================] - 74s 155ms/step - loss: 1.3160 - accuracy: 0.4912 - val_loss: 1.4563 - val_accuracy: 0.4967
Epoch 6/20
474/474 [==============================] - 69s 145ms/step - loss: 1.1109 - accuracy: 0.5837 - val_loss: 1.1619 - val_accuracy: 0.6283
Epoch 7/20
474/474 [==============================] - 69s 145ms/step - loss: 0.9522 - accuracy: 0.6561 - val_loss: 1.2317 - val_ac

# ResNet101

In [ ]:
from tensorflow.keras.applications import ResNet101

epochs = 15

modelRN101 = Sequential()

base_model = ResNet101(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelRN101.add(base_model)
modelRN101.add(Flatten())
modelRN101.add(Dense(512, activation='relu'))
modelRN101.add(Dense(20, activation='softmax'))

modelRN101.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

history = modelRN101.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelRN101.evaluate(testX, testY, batch_size=batch_size)

Epoch 1/15
474/474 [==============================] - 135s 255ms/step - loss: 2.8319 - accuracy: 0.1006 - val_loss: 2.7750 - val_accuracy: 0.0937
Epoch 2/15
474/474 [==============================] - 119s 251ms/step - loss: 2.3417 - accuracy: 0.1668 - val_loss: 2.2594 - val_accuracy: 0.1980
Epoch 3/15
474/474 [==============================] - 118s 248ms/step - loss: 2.1218 - accuracy: 0.2149 - val_loss: 2.1392 - val_accuracy: 0.2265
Epoch 4/15
474/474 [==============================] - 118s 248ms/step - loss: 2.0150 - accuracy: 0.2404 - val_loss: 2.3356 - val_accuracy: 0.1817
Epoch 5/15
474/474 [==============================] - 119s 251ms/step - loss: 1.8201 - accuracy: 0.2980 - val_loss: 1.8784 - val_accuracy: 0.3077
Epoch 6/15
474/474 [==============================] - 117s 248ms/step - loss: 1.6128 - accuracy: 0.3669 - val_loss: 2.4867 - val_accuracy: 0.2265
Epoch 7/15
474/474 [==============================] - 118s 248ms/step - loss: 1.4341 - accuracy: 0.4340 - val_loss: 1.6627 -

In [ ]:
from tensorflow.keras.applications import ResNet101

epochs = 15

modelRN101 = Sequential()

base_model = ResNet101(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelRN101.add(base_model)
modelRN101.add(Flatten())
modelRN101.add(Dense(512, activation='relu'))
modelRN101.add(Dense(20, activation='softmax'))

modelRN101.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

modelRN101.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 2048)              42658176  
                                                                 
 flatten_7 (Flatten)         (None, 2048)              0         
                                                                 
 dense_14 (Dense)            (None, 512)               1049088   
                                                                 
 dense_15 (Dense)            (None, 20)                10260     
                                                                 
Total params: 43,717,524
Trainable params: 43,612,180
Non-trainable params: 105,344
_________________________________________________________________


# ResNet101V2

In [ ]:
from tensorflow.keras.applications import ResNet101V2

epochs = 20

modelRN101 = Sequential()

base_model = ResNet101(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelRN101.add(base_model)
modelRN101.add(Flatten())
modelRN101.add(Dense(512, activation='relu'))
modelRN101.add(Dense(20, activation='softmax'))

modelRN101.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

history = modelRN101.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelRN101.evaluate(testX, testY, batch_size=batch_size)

Epoch 1/20
474/474 [==============================] - 162s 283ms/step - loss: 2.8904 - accuracy: 0.0884 - val_loss: 4.0076 - val_accuracy: 0.0543
Epoch 2/20
474/474 [==============================] - 121s 256ms/step - loss: 2.3392 - accuracy: 0.1735 - val_loss: 2.4681 - val_accuracy: 0.1663
Epoch 3/20
474/474 [==============================] - 127s 269ms/step - loss: 2.2973 - accuracy: 0.1763 - val_loss: 2.9889 - val_accuracy: 0.0512
Epoch 4/20
474/474 [==============================] - 118s 250ms/step - loss: 2.1644 - accuracy: 0.2092 - val_loss: 2.0345 - val_accuracy: 0.2399
Epoch 5/20
474/474 [==============================] - 119s 252ms/step - loss: 1.9520 - accuracy: 0.2634 - val_loss: 2.1705 - val_accuracy: 0.1931
Epoch 6/20
474/474 [==============================] - 118s 249ms/step - loss: 1.7925 - accuracy: 0.3088 - val_loss: 2.0821 - val_accuracy: 0.2826
Epoch 7/20
474/474 [==============================] - 119s 251ms/step - loss: 1.5634 - accuracy: 0.3868 - val_loss: 1.5903 -

# InceptionV3

In [ ]:
from tensorflow.keras.applications import InceptionV3

epochs = 20

modelIV3 = Sequential()

base_model = InceptionV3(include_top=False, weights=None, classes=num_classes, pooling=None)

modelIV3.add(Resizing(80, 80, interpolation="bilinear", input_shape=(32, 32, 3)))
modelIV3.add(base_model)
modelIV3.add(Flatten())
modelIV3.add(Dense(512, activation='relu'))
modelIV3.add(Dense(20, activation='softmax'))

modelIV3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

history = modelIV3.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelIV3.evaluate(testX, testY, batch_size=batch_size)

Epoch 1/20
474/474 [==============================] - 144s 275ms/step - loss: 2.3519 - accuracy: 0.1838 - val_loss: 3.1749 - val_accuracy: 0.1485
Epoch 2/20
474/474 [==============================] - 124s 262ms/step - loss: 1.6897 - accuracy: 0.3452 - val_loss: 1.4705 - val_accuracy: 0.4441
Epoch 3/20
474/474 [==============================] - 124s 262ms/step - loss: 1.1687 - accuracy: 0.5444 - val_loss: 1.2137 - val_accuracy: 0.5892
Epoch 4/20
474/474 [==============================] - 124s 262ms/step - loss: 0.8525 - accuracy: 0.6820 - val_loss: 1.9051 - val_accuracy: 0.5496
Epoch 5/20
474/474 [==============================] - 124s 262ms/step - loss: 0.6228 - accuracy: 0.7821 - val_loss: 0.7454 - val_accuracy: 0.7596
Epoch 6/20
474/474 [==============================] - 124s 262ms/step - loss: 0.4910 - accuracy: 0.8413 - val_loss: 0.8625 - val_accuracy: 0.7851
Epoch 7/20
474/474 [==============================] - 124s 262ms/step - loss: 0.3703 - accuracy: 0.8850 - val_loss: 0.4976 -

In [ ]:
from tensorflow.keras.applications import InceptionV3

epochs = 20

modelIV3 = Sequential()

base_model = InceptionV3(include_top=False, weights=None, classes=num_classes, pooling=None)

modelIV3.add(Resizing(80, 80, interpolation="bilinear", input_shape=(32, 32, 3)))
modelIV3.add(base_model)
modelIV3.add(Flatten())
modelIV3.add(Dense(512, activation='relu'))
modelIV3.add(Dense(20, activation='softmax'))

modelIV3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

modelIV3.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 80, 80, 3)         0         
                                                                 
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 flatten_8 (Flatten)         (None, 2048)              0         
                                                                 
 dense_16 (Dense)            (None, 512)               1049088   
                                                                 
 dense_17 (Dense)            (None, 20)                10260     
                                                                 
Total params: 22,862,132
Trainable params: 22,827,700
Non-trainable params: 34,432
_________________________________________________________________


# MobileNet

In [ ]:
from tensorflow.keras.applications import MobileNetV2

epochs = 20

modelMNV2 = Sequential()

base_model = MobileNetV2(include_top=False, weights=None, classes=num_classes, pooling=None)

modelMNV2.add(Resizing(72, 72, interpolation="bilinear", input_shape=(32, 32, 3)))
modelMNV2.add(base_model)
modelMNV2.add(Flatten())
modelMNV2.add(Dense(512, activation='relu'))
modelMNV2.add(Dense(20, activation='softmax'))

modelMNV2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
print(modelMNV2.summary())

history = modelMNV2.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelMNV2.evaluate(testX, testY, batch_size=batch_size)

Epoch 1/20
474/474 [==============================] - 920s 2s/step - loss: 2.7972 - accuracy: 0.0988 - val_loss: 3.0087 - val_accuracy: 0.0482
Epoch 2/20
474/474 [==============================] - 929s 2s/step - loss: 2.0092 - accuracy: 0.2460 - val_loss: 4.3072 - val_accuracy: 0.0488
Epoch 3/20
474/474 [==============================] - ETA: 0s - loss: 1.3546 - accuracy: 0.4648

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.applications import MobileNetV2

epochs = 20

modelMNV2 = Sequential()

base_model = MobileNetV2(include_top=False, weights=None, classes=num_classes, pooling=None)

modelMNV2.add(Resizing(72, 72, interpolation="bilinear", input_shape=(32, 32, 3)))
modelMNV2.add(base_model)
modelMNV2.add(Flatten())
modelMNV2.add(Dense(512, activation='relu'))
modelMNV2.add(Dense(20, activation='softmax'))

modelMNV2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

modelMNV2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_3 (Resizing)       (None, 72, 72, 3)         0         
                                                                 
 mobilenetv2_1.00_None (Func  (None, None, None, 1280)  2257984  
 tional)                                                         
                                                                 
 flatten_9 (Flatten)         (None, 5120)              0         
                                                                 
 dense_18 (Dense)            (None, 512)               2621952   
                                                                 
 dense_19 (Dense)            (None, 20)                10260     
                                                                 
Total params: 4,890,196
Trainable params: 4,856,084
Non-trainable params: 34,112
_______________________________________

# DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121
epochs = 20

modelDN121 = Sequential()

base_model = DenseNet121(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelDN121.add(base_model)
modelDN121.add(Flatten())
modelDN121.add(Dense(512, activation='relu'))
modelDN121.add(Dense(20, activation='softmax'))

modelDN121.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

history = modelDN121.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelDN121.evaluate(testX, testY, batch_size=batch_size)


Epoch 1/20
474/474 [==============================] - 106s 153ms/step - loss: 2.1873 - accuracy: 0.2284 - val_loss: 3.0202 - val_accuracy: 0.2172
Epoch 2/20
474/474 [==============================] - 85s 179ms/step - loss: 1.3342 - accuracy: 0.5208 - val_loss: 1.4055 - val_accuracy: 0.6051
Epoch 3/20
474/474 [==============================] - 80s 169ms/step - loss: 0.6827 - accuracy: 0.7963 - val_loss: 0.7492 - val_accuracy: 0.7933
Epoch 4/20
474/474 [==============================] - 96s 203ms/step - loss: 0.4426 - accuracy: 0.8812 - val_loss: 0.7105 - val_accuracy: 0.8309
Epoch 5/20
474/474 [==============================] - 78s 164ms/step - loss: 0.3354 - accuracy: 0.9131 - val_loss: 0.6079 - val_accuracy: 0.8502
Epoch 6/20
474/474 [==============================] - 66s 138ms/step - loss: 0.2607 - accuracy: 0.9335 - val_loss: 0.3166 - val_accuracy: 0.9285
Epoch 7/20
474/474 [==============================] - 80s 170ms/step - loss: 0.2365 - accuracy: 0.9406 - val_loss: 0.3293 - val_a

# DenseNet169

In [ ]:
from tensorflow.keras.applications import DenseNet169
epochs = 20

modelDN169 = Sequential()

base_model = DenseNet169(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelDN169.add(base_model)
modelDN169.add(Flatten())
modelDN169.add(Dense(512, activation='relu'))
modelDN169.add(Dense(20, activation='softmax'))

modelDN169.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
modelDN169.summary()

history = modelDN169.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelDN169.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 1664)              12642880  
                                                                 
 flatten_11 (Flatten)        (None, 1664)              0         
                                                                 
 dense_22 (Dense)            (None, 512)               852480    
                                                                 
 dense_23 (Dense)            (None, 20)                10260     
                                                                 
Total params: 13,505,620
Trainable params: 13,347,220
Non-trainable params: 158,400
_________________________________________________________________


In [ ]:
import time
from tensorflow.keras.applications import EfficientNetB0
epochs = 20

modelENB0 = Sequential()

base_model = EfficientNetB0(include_top=False, input_shape=(32, 32, 3), weights=None, classes=num_classes, pooling="max")

modelENB0.add(base_model)
modelENB0.add(Flatten())
modelENB0.add(Dense(512, activation='relu'))
modelENB0.add(Dense(20, activation='softmax'))

modelENB0.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy", "categorical_accuracy"])

modelENB0.summary()

history = modelENB0.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = modelENB0.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 flatten_12 (Flatten)        (None, 1280)              0         
                                                                 
 dense_24 (Dense)            (None, 512)               655872    
                                                                 
 dense_25 (Dense)            (None, 20)                10260     
                                                                 
Total params: 4,715,703
Trainable params: 4,673,680
Non-trainable params: 42,023
_________________________________________________________________
